In [3]:
import streamlit as st
import eurostat
import pandas as pd
import streamlit as st
from streamlit_option_menu import option_menu
from streamlit.components.v1 import html
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
import mysql.connector

# Full-time and part-time employment by sex, age and occupation (1 000) - Total

# # Extract


In [6]:

def get_data_FT_PT_EMP():
    toc_df = eurostat.get_toc_df()
    f = toc_df.loc[toc_df['code']=='LFSQ_EPGAIS']
    f = f.reset_index(drop=True)
    title = f['title'][0]
    
    dic = eurostat.get_dic('LFSQ_EPGAIS', 'worktime', full=False, frmt="dict", lang="en")
    dic1 = eurostat.get_dic('LFSQ_EPGAIS', 'isco08', full=False, frmt="dict", lang="en")
    dic2 = eurostat.get_dic('LFSQ_EPGAIS', 'sex', full=False, frmt="dict", lang="en")
    dic3 = eurostat.get_dic('LFSQ_EPGAIS', 'age', full=False, frmt="dict", lang="en")
    df = pd.read_csv('geo.csv')
    
    par_values = eurostat.get_dic('LFSQ_EPGAIS', 'geo',full=False, frmt="dict", lang="en")
    
    #my_filter_pars = {'startPeriod': 1960, 'geo': list(df['code'])}
    my_filter_pars = {'startPeriod': 2010,'endPeriod': 2022, 'geo': list(df['code'])}
    
    data = eurostat.get_data_df('LFSQ_EPGAIS', filter_pars=my_filter_pars)
    
    data['geo\TIME_PERIOD'] = data['geo\TIME_PERIOD'].replace(par_values)
    data['worktime'] = data['worktime'].replace(dic)
    data['isco08'] = data['isco08'].replace(dic1)
    data['sex'] = data['sex'].replace(dic2)
    data['age'] = data['age'].replace(dic3)
    data = data.drop(['freq','unit'], axis=1)
    
    df = data.melt(id_vars=["worktime","sex","age",'isco08',"geo\TIME_PERIOD"], 
        var_name="Year", 
        value_name="Value")

    df = df.dropna()
    
    #dfn = df.convert_dtypes()
    
    return df

# # Transform

In [7]:
df_FT_PT_EMP = get_data_FT_PT_EMP()
# Splitting the column into two columns
df_FT_PT_EMP[['Year', 'Quarter']] = df_FT_PT_EMP['Year'].str.split('-', expand=True)
df_FT_PT_EMP.rename(columns={'geo\TIME_PERIOD': 'geo'}, inplace=True)
df_FT_PT_EMP


,worktime,sex,age,isco08,geo,Year,Value,Quarter
0,Full-time,Females,From 15 to 24 years,No response,Germany,2010,30.7,Q1
1,Part-time,Females,From 15 to 24 years,No response,Germany,2010,45.7,Q1
2,Total,Females,From 15 to 24 years,No response,Germany,2010,76.4,Q1
9,Full-time,Females,From 15 to 24 years,Professionals,Germany,2010,47.8,Q1
10,Part-time,Females,From 15 to 24 years,Professionals,Germany,2010,33.1,Q1
...,...,...,...,...,...,...,...,...
1783269,Total,Total,75 years or over,Service and sales workers,European Union - 27 countries (from 2020),2022,105.8,Q4
1783272,Total,Total,75 years or over,"Skilled agricultural, forestry and fishery wor...",European Union - 27 countries (from 2020),2022,67.4,Q4
1783284,Full-time,Total,75 years or over,Total,European Union - 27 countries (from 2020),2022,210.9,Q4
1783286,Part-time,Total,75 years or over,Total,European Union - 27 countries (from 2020),2022,436.8,Q4


# # Load

In [10]:
mydb = mysql.connector.connect(host="localhost", user="root", password="research_lab_2022", database="data_story")
print(mydb)

menu = mydb.cursor()

# Truncate the table to remove existing data
menu.execute("TRUNCATE TABLE ft_pt_emp")

for index, row in df_FT_PT_EMP.iterrows():
     menu.execute("INSERT INTO ft_pt_emp (worktime, sex, age,isco08, geo, Year,Value,Quarter) values(%s, %s, %s, %s, %s, %s, %s, %s)", (row.worktime, row.sex, row.age, row.isco08, row.geo, row.Year, row.Value, row.Quarter))

mydb.commit()
menu.close()

True

# Unemployment rate by sex

## Extract

In [5]:

def get_data_UNEMP_SEX():
    toc_df = eurostat.get_toc_df()
    f = toc_df.loc[toc_df['code']=='TESEM120']
    f = f.reset_index(drop=True)
    title = f['title'][0]
    
    #dic = eurostat.get_dic('TESEM120', 'worktime', full=False, frmt="dict", lang="en")
    dic1 = eurostat.get_dic('TESEM120', 'unit', full=False, frmt="dict", lang="en")
    dic2 = eurostat.get_dic('TESEM120', 'sex', full=False, frmt="dict", lang="en")
    dic3 = eurostat.get_dic('TESEM120', 'age', full=False, frmt="dict", lang="en")
    df = pd.read_csv('geo.csv')
    
    par_values = eurostat.get_dic('TESEM120', 'geo',full=False, frmt="dict", lang="en")
    
    #my_filter_pars = {'startPeriod': 1960, 'geo': list(df['code'])}
    my_filter_pars = {'startPeriod': 2010,'endPeriod': 2022, 'geo': list(df['code'])}
    
    data = eurostat.get_data_df('TESEM120', filter_pars=my_filter_pars)
    
    data['geo\TIME_PERIOD'] = data['geo\TIME_PERIOD'].replace(par_values)
    #data['worktime'] = data['worktime'].replace(dic)
    #data['unit'] = data['unit'].replace(dic1)
    data['sex'] = data['sex'].replace(dic2)
    data['age'] = data['age'].replace(dic3)
    data = data.drop(['freq','unit'], axis=1)
    
    df = data.melt(id_vars=["sex","age","geo\TIME_PERIOD"], 
        var_name="Year", 
        value_name="Value")

    df = df.dropna()
    
    #dfn = df.convert_dtypes()
    
    return df

In [6]:
df_UNEMP_SEX = get_data_UNEMP_SEX()
df_UNEMP_SEX.rename(columns={'geo\TIME_PERIOD': 'geo'}, inplace=True)
df_UNEMP_SEX


,sex,age,geo,Year,Value
0,Females,From 15 to 74 years,Germany,2010,6.1
1,Males,From 15 to 74 years,Germany,2010,7.0
2,Total,From 15 to 74 years,Germany,2010,6.6
3,Females,From 15 to 74 years,Austria,2010,4.9
4,Males,From 15 to 74 years,Austria,2010,5.5
...,...,...,...,...,...
1087,Males,From 15 to 74 years,Sweden,2022,7.0
1088,Total,From 15 to 74 years,Sweden,2022,7.5
1089,Females,From 15 to 74 years,European Union - 27 countries (from 2020),2022,6.5
1090,Males,From 15 to 74 years,European Union - 27 countries (from 2020),2022,5.9


In [7]:
mydb = mysql.connector.connect(host="localhost", user="root", password="research_lab_2022", database="data_story")
print(mydb)

menu = mydb.cursor()

# Truncate the table to remove existing data
menu.execute("TRUNCATE TABLE unemp_sex")

for index, row in df_UNEMP_SEX.iterrows():
     menu.execute("INSERT INTO unemp_sex (sex, age, geo, Year,Value) values(%s, %s, %s, %s, %s)", (row.sex, row.age, row.geo, row.Year, row.Value))

mydb.commit()
menu.close()

True

In [8]:

def get_data_EMP_SEX():
    toc_df = eurostat.get_toc_df()
    f = toc_df.loc[toc_df['code']=='TESEM010']
    f = f.reset_index(drop=True)
    title = f['title'][0]
    
    dic = eurostat.get_dic('TESEM010', 'indic_em', full=False, frmt="dict", lang="en")
    dic1 = eurostat.get_dic('TESEM010', 'unit', full=False, frmt="dict", lang="en")
    dic2 = eurostat.get_dic('TESEM010', 'sex', full=False, frmt="dict", lang="en")
    dic3 = eurostat.get_dic('TESEM010', 'age', full=False, frmt="dict", lang="en")
    df = pd.read_csv('geo.csv')
    
    par_values = eurostat.get_dic('TESEM010', 'geo',full=False, frmt="dict", lang="en")
    
    #my_filter_pars = {'startPeriod': 1960, 'geo': list(df['code'])}
    my_filter_pars = {'startPeriod': 2010,'endPeriod': 2022, 'geo': list(df['code'])}
    
    data = eurostat.get_data_df('TESEM010', filter_pars=my_filter_pars)
    
    data['geo\TIME_PERIOD'] = data['geo\TIME_PERIOD'].replace(par_values)
    data['indic_em'] = data['indic_em'].replace(dic)
    #data['unit'] = data['unit'].replace(dic1)
    data['sex'] = data['sex'].replace(dic2)
    data['age'] = data['age'].replace(dic3)
    data = data.drop(['freq','unit','indic_em'], axis=1)
    
    df = data.melt(id_vars=["sex","age","geo\TIME_PERIOD"], 
        var_name="Year", 
        value_name="Value")

    df = df.dropna()
    
    #dfn = df.convert_dtypes()
    
    return df

In [9]:
df_EMP_SEX = get_data_EMP_SEX()
df_EMP_SEX.rename(columns={'geo\TIME_PERIOD': 'geo'}, inplace=True)
df_EMP_SEX


,sex,age,geo,Year,Value
0,Females,From 20 to 64 years,Germany,2010,68.8
1,Males,From 20 to 64 years,Germany,2010,79.3
2,Total,From 20 to 64 years,Germany,2010,74.0
3,Females,From 20 to 64 years,Austria,2010,68.8
4,Males,From 20 to 64 years,Austria,2010,79.0
...,...,...,...,...,...
1087,Males,From 20 to 64 years,Sweden,2022,85.0
1088,Total,From 20 to 64 years,Sweden,2022,82.2
1089,Females,From 20 to 64 years,European Union - 27 countries (from 2020),2022,69.3
1090,Males,From 20 to 64 years,European Union - 27 countries (from 2020),2022,80.0


In [10]:
mydb = mysql.connector.connect(host="localhost", user="root", password="research_lab_2022", database="data_story")
print(mydb)

menu = mydb.cursor()

# Truncate the table to remove existing data
menu.execute("TRUNCATE TABLE emp_sex")

for index, row in df_EMP_SEX.iterrows():
     menu.execute("INSERT INTO emp_sex (sex, age, geo, Year, Value) values(%s, %s, %s, %s, %s)", (row.sex, row.age, row.geo, row.Year, row.Value))

mydb.commit()
menu.close()

True